In [1]:
from typing import Tuple, List
import polars as pl
from pprint import pprint
import sys
from pathlib import Path
import psutil
import pandas as pd
import time
import numpy as np

In [2]:
# 데이터 로드
df = pl.scan_parquet('../data/maude_sample.parquet')
total_rows = df.select(pl.len()).collect().item()
total_cols = len(df.collect_schema().names())
print(f"전체 행: {total_rows:,}개, 전체 컬럼: {total_cols}개")

전체 행: 14,132,321개, 전체 컬럼: 2247개


In [3]:
# 1차 : 중복 제거 컬럼 
dedup_cols = [
    'report_number',
    'date_of_event', 
    'device_0_manufacturer_d_name',
    'device_0_udi_di',
    'device_0_lot_number',
    'device_0_udi_public'
]

# Unknown / N/A 패턴 리스트
na_patterns = r'^None$|^UNK|NOT APPLICABLE|NOT REPORTED|^N/A$|^NA$|^$|\s+$|^UNKNOWN$|^NI$|^NULL$'



In [4]:
# 일단 중복된 행들만 확인
# 조합 (6개 컬럼)의 개수
df_with_cnt = df.with_columns(
    pl.len().over(dedup_cols).alias('duplicate_cnt')
)

# cnt가 2 이상이 경우에만 
# cnt가 1인 경우에는 삭제할 필요가 없으므로
df_duplicates_only = df_with_cnt.filter(
    pl.col('duplicate_cnt') >= 2
)

duplicate_cnt = df_duplicates_only.select(pl.len()).collect().item()
print(f"중복된 행의 개수: {duplicate_cnt:,}개")

unique_cnt = df.unique(subset=dedup_cols, maintain_order=True).select(pl.len()).collect().item()
print(f"유일한 행의 개수: {unique_cnt:,}개")

중복된 행의 개수: 39,177개
유일한 행의 개수: 14,110,242개


In [5]:
sample = df_duplicates_only.select(
    dedup_cols + ['duplicate_cnt']
).head(10).collect()

print(sample)

shape: (10, 7)
┌──────────────┬─────────────┬─────────────┬─────────────┬─────────────┬─────────────┬─────────────┐
│ report_numbe ┆ date_of_eve ┆ device_0_ma ┆ device_0_ud ┆ device_0_lo ┆ device_0_ud ┆ duplicate_c │
│ r            ┆ nt          ┆ nufacturer_ ┆ i_di        ┆ t_number    ┆ i_public    ┆ nt          │
│ ---          ┆ ---         ┆ d_name      ┆ ---         ┆ ---         ┆ ---         ┆ ---         │
│ str          ┆ str         ┆ ---         ┆ str         ┆ str         ┆ str         ┆ u32         │
│              ┆             ┆ str         ┆             ┆             ┆             ┆             │
╞══════════════╪═════════════╪═════════════╪═════════════╪═════════════╪═════════════╪═════════════╡
│ 0009613348-2 ┆ 20240813    ┆ INSTITUT    ┆ 07630031719 ┆ null        ┆ 07630031719 ┆ 2           │
│ 024-012765   ┆             ┆ STRAUMANN   ┆ 232         ┆             ┆ 232         ┆             │
│              ┆             ┆ AG          ┆             ┆             ┆    

# 함수 정의

In [6]:
def remove_na_values(df, dedup_cols, na_patterns, verbose=True):
    """
    Na / Unknwon 값이 있는 행을 제거하는 함수
    
    작동방식 :
    1. 각 컬럼에 대해 유효한 값인지 체크
    2. 모든 조건은 포함한(모두 만족하는) 행으로 필터링
    3. 필터 적용

    Parameters:
    df : polars.DataFrame
        원본 DF
    dedup_cols : list
        -> 모두 유효해야 함
    na_patterns : str
        NA / Unknown 패턴 정규식
    verbose : bool
        -> 현재 진행상황 출력해서 확인할 수 있게

    Returns: 
    polars.LazyFrame
        비어진 값이나 모르는 값이 없는 DF
    """

    # 진행상확 확인
    if verbose:
        print("na 값 제거")
        print(f"패턴: {na_patterns}")

    # 제거 되기 전 개수 확인
    before_cnt = df.select(pl.len()).collect().item()
    if verbose:
        print(f"제거 전 행 개수 : {before_cnt:,}개")
    
    # ===
    # 각 컬럼별로 필터 조건
    # ===
    conditions = []

    for col in dedup_cols:
        # 컬럼이 존재하는지 확인
        if col in df.collect_schema().names():
            # 유효한 값의 조건
            # null / na_patterns 패턴에 매칭되지 않는 값
            cond = (
                pl.col(col).is_not_null()
                &
                ~ pl.col(col).cast(pl.Utf8).str.to_uppercase().str.contains(na_patterns)
                # ~ 은 not 연산자
            )
            conditions.append(cond)

            if verbose:
                print(f"컬럼 '{col}'에 대해 na/unknown 값 제거 조건 추가")
        else:
            if verbose:
                print(f"컬럼 '{col}'이(가) 존재하지 않음. 건너뜀")
        
# 예외처리
    if not conditions:
        if verbose:
            print("제거할 조건이 없음. 원본 DF 반환")
        return df

# ===
# 모든 조건을 and 조건으로 결합
# (모든 조건을 만족해야 함)
# ===
    final_condition = conditions[0]
    for cond in conditions[1:]:
        final_condition = final_condition & cond

# 필터 적용
    df_cleaned = df.filter(final_condition)

# 결과
    after_cnt = df_cleaned.select(pl.len()).collect().item()
    removed_cnt = before_cnt - after_cnt

    if verbose:
        print(f"제거 후 행 개수 : {after_cnt:,}개")
        print(f"제거된 행 개수 : {removed_cnt:,}개")

        return df_cleaned

In [7]:
def analyze_duplicates(df, group_cols, verbose = True):
    """
    중복 데이터 분석 함수들

    작동 방식 
    1. 전체 개수 확인
    2. 고유(unique) 개수 확인
    3. 전체 - 고유 = 중복 개수 확인

    Parameters:
    df : polars.DataFrame -> 원본
    dedup_cols : list -> 중복 확인 컬럼
    verbose : bool -> 진행상황 출력 여부

    returns:

    tuple : (전체 개수, 고유 개수, 중복 개수)
    """

    if verbose:
        print(f" 중복 확인")

    # 전체 개수
    total_cnt = df.select(pl.len()).collect().item()

    # 고유 개수
    unique_cnt = df.unique(
        subset = group_cols, # 중복 ㅎ판단
        maintain_order = True
    ).select(pl.len()).collect().item()

    # 중복 개수
    duplicate_cnt = total_cnt - unique_cnt

    if verbose:
        print(f"전체 개수 : {total_cnt:,}개")
        print(f"고유 개수 : {unique_cnt:,}개")
        print(f"중복 개수 : {duplicate_cnt:,}개")
        for i, col in enumerate(dedup_cols, start=1):
            print(f"{i}. {col}")

    return total_cnt, unique_cnt, duplicate_cnt

In [8]:
def remove_duplicates(df, dedup_cols, keep = 'first', verbose = True):
    """
    중복 데이터 제거 함수

    작동 방식
    1. dedup_cols 기준으로 중복 판단
    2. keep 옵션에 따라 첫번째/마지막 행 유지
    3. 중복 제거된 DF 반환

    Parameters:
    df : polars.DataFrame -> 원본 DF
    dedup_cols : list -> 중복 판단 컬럼 리스트
    keep : str -> 'first' or 'last'
        'first' : 첫번째 행 유지
        'last' : 마지막 행 유지
    verbose : bool -> 진행상황 출력 여부

    Returns:
    polars.DataFrame -> 중복 제거된 DF
    """

    if verbose:
        print("중복 제거 시작")
        print(f"중복 판단 컬럼: {dedup_cols}")
        print(f"유지 옵션: {keep}")

    # 중복 제거
    df_deduped = df.unique(
        subset = dedup_cols,
        maintain_order = True,
        keep = 'first'
    )

    if verbose:
        before_cnt = df.select(pl.len()).collect().item()
        after_cnt = df_deduped.select(pl.len()).collect().item()
        removed_cnt = before_cnt - after_cnt

        print(f"제거 전 행 개수 : {before_cnt:,}개")
        print(f"제거 후 행 개수 : {after_cnt:,}개")
        print(f"제거된 행 개수 : {removed_cnt:,}개")

    return df_deduped

### 함수 실행

In [9]:
# na 값들 제거
df_cleaned = remove_na_values(df_duplicates_only, dedup_cols, na_patterns)
df_cleaned

na 값 제거
패턴: ^None$|^UNK|NOT APPLICABLE|NOT REPORTED|^N/A$|^NA$|^$|\s+$|^UNKNOWN$|^NI$|^NULL$
제거 전 행 개수 : 39,177개
컬럼 'report_number'에 대해 na/unknown 값 제거 조건 추가
컬럼 'date_of_event'에 대해 na/unknown 값 제거 조건 추가
컬럼 'device_0_manufacturer_d_name'에 대해 na/unknown 값 제거 조건 추가
컬럼 'device_0_udi_di'에 대해 na/unknown 값 제거 조건 추가
컬럼 'device_0_lot_number'에 대해 na/unknown 값 제거 조건 추가
컬럼 'device_0_udi_public'에 대해 na/unknown 값 제거 조건 추가
제거 후 행 개수 : 17,480개
제거된 행 개수 : 21,697개


In [10]:
total, unique, duplicate = analyze_duplicates(df_cleaned, dedup_cols)
pprint((total, unique, duplicate))  

 중복 확인


전체 개수 : 17,480개
고유 개수 : 8,201개
중복 개수 : 9,279개
1. report_number
2. date_of_event
3. device_0_manufacturer_d_name
4. device_0_udi_di
5. device_0_lot_number
6. device_0_udi_public
(17480, 8201, 9279)


In [11]:
df_final = remove_duplicates(df_cleaned, dedup_cols, keep='first')
df_final

중복 제거 시작
중복 판단 컬럼: ['report_number', 'date_of_event', 'device_0_manufacturer_d_name', 'device_0_udi_di', 'device_0_lot_number', 'device_0_udi_public']
유지 옵션: first
제거 전 행 개수 : 17,480개
제거 후 행 개수 : 8,201개
제거된 행 개수 : 9,279개


In [12]:
final_count = df_final.select(pl.len()).collect().item()
final_count

8201